# This notebook implements the BOW model with Ordinal Output Categories 

In [9]:
import pandas as pd 
import numpy as np
import matplotlib as plt 
import glob
from importlib import reload

import os, sys, re, json, time, datetime, shutil
from common import utils, constants, spell

import tensorflow as tf
import tripadvisor_ds
import visualization

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

## 1. Load tripadvisor data 

In [46]:

reload(tripadvisor_ds)

input_length = 500
max_bytes = 2**31 - 1

data_file = 'data/tripadvisor_ds.pkl'

if os.path.isfile(data_file):

    bytes_in = bytearray(0)
    input_size = os.path.getsize(data_file)
    with open(data_file, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    ds = pickle.loads(bytes_in)
        
else:
    ds = tripadvisor_ds.TripAdvisor_DS().process(input_length=input_length)
    ds.save(data_file)


In [47]:
# -----------------------------------------
# convert the output to Ordinal Categories 
# 2-star rating: [0,0,0]
# 3-star rating: [1,0,0]
# 4-star rating: [1,1,0]
# 5-star rating: [1,1,1]
# -----------------------------------------

ds.get_ord_labels()

In [50]:
labels = pd.DataFrame({'rating': ds.train_labels})
labels.rating.value_counts()

3    413463
2    231827
1    106079
0     90053
Name: rating, dtype: int64

## 2. Build the Model

In [135]:
# set model parameters 
model_params = dict(V=ds.vocab.size, 
                    embed_dim=100, 
                    num_classes=len(ds.target_labels),
                    encoder_type='bow', 
                    hidden_dims=[1024, 64], 
                    input_length=input_length,
                    lr=0.0001, 
                    optimizer='adam', 
                    beta=0.00001)
                    
train_params = dict(batch_size=64, 
                    total_epochs=20, 
                    eval_every=2)


summary_params = dict(chkpt_dir="./tmp/266_bow_ord" + datetime.datetime.now().strftime("%Y%m%d-%H%M"))

In [137]:
# setup tensorboard 

if os.path.isdir(summary_params['chkpt_dir']):
    shutil.rmtree(summary_params['chkpt_dir'])

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [138]:
def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    # prepare vocabulary  
    W_embed_ = tf.get_variable("W_embed", shape=[V, embed_dim], \
                               initializer=tf.random_uniform_initializer(-init_scale, init_scale), \
                               trainable=True)
        
    # look up word embedding 
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_, name="embed_x")
        
    return xs_

def fully_connected_layers(h0_, hidden_dims, activation=tf.nn.relu,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        h_ = tf.layers.dense(h_, hdim, activation=activation, name=("Hidden_%d"%i))
        if dropout_rate > 0:
            h_ = tf.layers.dropout(h_, rate=dropout_rate, training=is_training )

    return h_

def softmax_output_layer(h_, labels_, num_classes):
    
    W_out_ = tf.get_variable("W_out",  shape=[h_.get_shape().as_list()[1], num_classes], \
                               initializer=tf.random_normal_initializer())
    b_out_ = tf.get_variable("b_out", shape=[num_classes])

    logits_ = tf.add(tf.matmul(h_, W_out_), b_out_)
        
    if labels_ is None:
        return None, logits_
    
    with tf.variable_scope("Softmax_Layer"):

        softmax_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_, logits=logits_)
        
        loss_ = tf.reduce_mean(softmax_)
    
    return loss_, logits_


def sigmoid_output_layer(h_, labels_, num_classes):
    
    W_out_ = tf.get_variable("W_out",  shape=[h_.get_shape().as_list()[1], num_classes], \
                               initializer=tf.random_normal_initializer())
    b_out_ = tf.get_variable("b_out", shape=[num_classes])

    logits_ = tf.add(tf.matmul(h_, W_out_), b_out_)
        
    if labels_ is None:
        return None, logits_
    
    with tf.variable_scope("Sigmoid_Layer"):
        
        sigmoid_ = tf.nn.sigmoid(logits_)
        
        loss_ = num_classes * tf.reduce_mean(tf.squared_difference( labels_, sigmoid_))
        
    return loss_, logits_


def BOW(ids_, V, embed_dim, hidden_dims, dropout_rate=0, is_training=None):
    assert is_training is not None, "is_training must be explicitly set to True or False"

    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim)
     
    sum_xs_ = tf.reduce_sum(xs_, 1)

    h_ = fully_connected_layers(sum_xs_, hidden_dims, \
                           dropout_rate=dropout_rate, is_training=is_training)
    return h_, xs_


def conv_net(ids_, V, embed_dim, filter_sizes, num_filters, hidden_dims, input_length, dropout_rate=0, is_training=None):

    assert is_training is not None, "is_training must be explicitly set to True or False"

    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim)

    xs_ = tf.expand_dims(xs_, -1)
        
    pooled_outputs_ = []
    for _, filter_size in enumerate(filter_sizes):
        with tf.name_scope("Conv_MaxPool_%d"%filter_size):
            
            # Convolution Layer
            filter_shape = [filter_size, embed_dim, 1, num_filters]
            W_ = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b_ = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv_ = tf.nn.conv2d(
                xs_,
                W_,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            
            # Activation
            h_ = tf.nn.relu(tf.nn.bias_add(conv_, b_), name="relu")
            
            # Maxpooling 
            pooled_ = tf.nn.max_pool(
                h_,
                ksize=[1, input_length - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],
                padding='VALID',
                name="pool")
            pooled_outputs_.append(pooled_)
            
            variable_summaries(pooled_)

    # Combine all the pooled features and flatten it
    num_filters_total = num_filters * len(filter_sizes)
    h_ = tf.concat(pooled_outputs_, 3)
    h_ = tf.reshape(h_, [-1, num_filters_total])
    
    # fully connected layers
    with tf.variable_scope("FC_Layer"):
        h_ = fully_connected_layers(h_, hidden_dims, is_training = is_training)

    return h_, xs_


In [139]:
tf.reset_default_graph() 

X = tf.placeholder(tf.int32, [None, input_length], name='input_x')
Y = tf.placeholder(tf.float32, [None, 3], name='input_y')
    
if model_params['encoder_type'] == 'bow':
    h_, xs_ = BOW(X, model_params['V'], 
                      model_params['embed_dim'],  
                      model_params['hidden_dims'],
                      is_training=True)


with tf.variable_scope("Output_Layer"):
    loss_, logits_ = sigmoid_output_layer(h_, Y, model_params['num_classes'] -1)
    

with tf.name_scope("Prediction"):
    pred_proba_ = tf.nn.sigmoid(logits_, name="pred_proba")
    
    pred_max_ = tf.reduce_sum(tf.cast(tf.great(pred_proba_ ,0.5), tf.int32), axis = 1, name="pred_max")
    
    predictions_dict = {"proba": pred_proba_, "max": pred_max_}

with tf.variable_scope("Regularization"):
    l2_penalty_ = tf.nn.l2_loss(xs_)  # l2 loss on embeddings
    for var_ in tf.trainable_variables():
        if "Embedding_Layer" in var_.name:
            continue
        l2_penalty_ += tf.nn.l2_loss(var_)
    l2_penalty_ *= model_params['beta']  # scale by regularization strength
    tf.summary.scalar('l2_penalty', l2_penalty_)
    regularized_loss_ = loss_ + l2_penalty_
    tf.summary.scalar('regularized_loss', regularized_loss_)

with tf.variable_scope("Training"):
    if model_params['optimizer'] == 'adagrad':
        optimizer_ = tf.train.AdagradOptimizer(model_params['lr'])
    elif  model_params['optimizer'] == 'adam':
        optimizer_ = tf.train.AdamOptimizer(model_params['lr'])
    else:
        optimizer_ = tf.train.GradientDescentOptimizer(model_params['lr'])
    train_op_ = optimizer_.minimize(regularized_loss_,
                    global_step=tf.train.get_global_step())


with tf.name_scope("Evaluation"):

    correct_pred_ = tf.equal(tf.cast(pred_max_, tf.int32), tf.cast(tf.reduce_sum(Y, 1), tf.int32))
    accuracy_ = tf.reduce_mean(tf.cast(correct_pred_, tf.float32))

    tf.summary.scalar('loss', loss_)
    tf.summary.scalar('accuracy', accuracy_)



## 3. Train the Model 

In [141]:

# start session
sess = tf.Session()
graph = tf.get_default_graph()

# Tensorboard - Visualize graph 
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(summary_params['chkpt_dir'] + '/train', sess.graph)
test_writer = tf.summary.FileWriter(summary_params['chkpt_dir'] + '/test')

print("tensorboard --logdir={}/train".format(summary_params['chkpt_dir']))
print("tensorboard --logdir={}/test".format(summary_params['chkpt_dir']))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

# Run the initializer
sess.run(init)
sess.run(init_l)

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)



for i in range(train_params['total_epochs']):
    t0 = time.time()

    train_batches = 1
    train_accuracy = 0.0
    
    for (bx, by) in utils.multi_batch_generator(train_params['batch_size'], \
                                        ds.padded_train_features, ds.train_ord_labels):

        summary, batch_loss, _, batch_accuracy, pred_proba, pred_max = sess.run(
            [merged, regularized_loss_, train_op_, accuracy_, pred_proba_, pred_max_], feed_dict={X: bx, Y: by})
        
        #print(pred_proba)
        #print(pred_max)
        
        train_batches +=1
        train_accuracy += batch_accuracy
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean

        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 25 == 0):
            print("{:5,} examples, moving-average loss {:.2f}, train accuracy {:.2f}"\
                  .format(total_examples, loss_ema, train_accuracy/train_batches))    
            
        train_writer.add_summary(summary, total_batches)

    print("Completed {} epoch in {:s}".format(i, utils.pretty_timedelta(since=t0)))
    
    train_accuracy = train_accuracy/train_batches
    print("Train accurary:{:.5f}".format(train_accuracy))
    
    
    # run the validation dataset 
    validate_batches = 1
    validate_accuracy = 0.0
    for (vx, vy) in utils.multi_batch_generator(train_params['batch_size'], \
                                            ds.padded_validate_features, ds.validate_ord_labels):

        summary, batch_accuracy = sess.run([merged, accuracy_], feed_dict={X: vx, Y: vy})

        validate_batches +=1
        validate_accuracy += batch_accuracy

        test_writer.add_summary(summary, total_batches + validate_batches)

    validate_accuracy = validate_accuracy/validate_batches
    print("Validate accuracy:{:.5f}".format(validate_accuracy))
        

tensorboard --logdir=./tmp/266_cnn_20181207-2305/train
tensorboard --logdir=./tmp/266_cnn_20181207-2305/test
1,600 examples, moving-average loss 0.54, train accuracy 0.39
3,200 examples, moving-average loss 0.45, train accuracy 0.45
4,800 examples, moving-average loss 0.38, train accuracy 0.50
6,400 examples, moving-average loss 0.34, train accuracy 0.53
8,000 examples, moving-average loss 0.32, train accuracy 0.54
9,600 examples, moving-average loss 0.30, train accuracy 0.56
11,200 examples, moving-average loss 0.29, train accuracy 0.57
12,800 examples, moving-average loss 0.28, train accuracy 0.59
14,400 examples, moving-average loss 0.27, train accuracy 0.60
16,000 examples, moving-average loss 0.28, train accuracy 0.60
17,600 examples, moving-average loss 0.27, train accuracy 0.61
19,200 examples, moving-average loss 0.27, train accuracy 0.62
20,800 examples, moving-average loss 0.29, train accuracy 0.62
22,400 examples, moving-average loss 0.29, train accuracy 0.62
24,000 examples

206,400 examples, moving-average loss 0.24, train accuracy 0.69
208,000 examples, moving-average loss 0.24, train accuracy 0.69
209,600 examples, moving-average loss 0.23, train accuracy 0.69
211,200 examples, moving-average loss 0.25, train accuracy 0.69
212,800 examples, moving-average loss 0.23, train accuracy 0.69
214,400 examples, moving-average loss 0.24, train accuracy 0.69
216,000 examples, moving-average loss 0.25, train accuracy 0.69
217,600 examples, moving-average loss 0.24, train accuracy 0.69
219,200 examples, moving-average loss 0.24, train accuracy 0.69
220,800 examples, moving-average loss 0.24, train accuracy 0.69
222,400 examples, moving-average loss 0.23, train accuracy 0.69
224,000 examples, moving-average loss 0.24, train accuracy 0.69
225,600 examples, moving-average loss 0.24, train accuracy 0.69
227,200 examples, moving-average loss 0.24, train accuracy 0.69
228,800 examples, moving-average loss 0.24, train accuracy 0.69
230,400 examples, moving-average loss 0.

412,800 examples, moving-average loss 0.24, train accuracy 0.70
414,400 examples, moving-average loss 0.23, train accuracy 0.70
416,000 examples, moving-average loss 0.24, train accuracy 0.70
417,600 examples, moving-average loss 0.24, train accuracy 0.70
419,200 examples, moving-average loss 0.22, train accuracy 0.70
420,800 examples, moving-average loss 0.24, train accuracy 0.70
422,400 examples, moving-average loss 0.23, train accuracy 0.70
424,000 examples, moving-average loss 0.24, train accuracy 0.70
425,600 examples, moving-average loss 0.22, train accuracy 0.70
427,200 examples, moving-average loss 0.23, train accuracy 0.70
428,800 examples, moving-average loss 0.23, train accuracy 0.70
430,400 examples, moving-average loss 0.24, train accuracy 0.70
432,000 examples, moving-average loss 0.24, train accuracy 0.70
433,600 examples, moving-average loss 0.23, train accuracy 0.70
435,200 examples, moving-average loss 0.24, train accuracy 0.70
436,800 examples, moving-average loss 0.

619,200 examples, moving-average loss 0.23, train accuracy 0.70
620,800 examples, moving-average loss 0.23, train accuracy 0.70
622,400 examples, moving-average loss 0.23, train accuracy 0.70
624,000 examples, moving-average loss 0.23, train accuracy 0.70
625,600 examples, moving-average loss 0.22, train accuracy 0.70
627,200 examples, moving-average loss 0.22, train accuracy 0.70
628,800 examples, moving-average loss 0.22, train accuracy 0.70
630,400 examples, moving-average loss 0.22, train accuracy 0.70
632,000 examples, moving-average loss 0.23, train accuracy 0.70
633,600 examples, moving-average loss 0.23, train accuracy 0.70
635,200 examples, moving-average loss 0.21, train accuracy 0.70
636,800 examples, moving-average loss 0.23, train accuracy 0.70
638,400 examples, moving-average loss 0.23, train accuracy 0.70
640,000 examples, moving-average loss 0.22, train accuracy 0.70
641,600 examples, moving-average loss 0.24, train accuracy 0.70
643,200 examples, moving-average loss 0.

825,600 examples, moving-average loss 0.23, train accuracy 0.71
827,200 examples, moving-average loss 0.24, train accuracy 0.71
828,800 examples, moving-average loss 0.22, train accuracy 0.71
830,400 examples, moving-average loss 0.23, train accuracy 0.71
832,000 examples, moving-average loss 0.23, train accuracy 0.71
833,600 examples, moving-average loss 0.22, train accuracy 0.71
835,200 examples, moving-average loss 0.21, train accuracy 0.71
836,800 examples, moving-average loss 0.22, train accuracy 0.71
838,400 examples, moving-average loss 0.22, train accuracy 0.71
840,000 examples, moving-average loss 0.21, train accuracy 0.71
Completed 0 epoch in 0:12:31
Train accurary:0.70570
Validate accuracy:0.71522
841,550 examples, moving-average loss 0.23, train accuracy 0.47
843,150 examples, moving-average loss 0.22, train accuracy 0.67
844,750 examples, moving-average loss 0.22, train accuracy 0.70
846,350 examples, moving-average loss 0.22, train accuracy 0.71
847,950 examples, moving-a

...

1,825,500 examples, moving-average loss 0.21, train accuracy 0.73
1,827,100 examples, moving-average loss 0.21, train accuracy 0.73
1,828,700 examples, moving-average loss 0.20, train accuracy 0.73
1,830,300 examples, moving-average loss 0.21, train accuracy 0.73
1,831,900 examples, moving-average loss 0.21, train accuracy 0.73
1,833,500 examples, moving-average loss 0.21, train accuracy 0.73
1,835,100 examples, moving-average loss 0.22, train accuracy 0.73
1,836,700 examples, moving-average loss 0.21, train accuracy 0.73
1,838,300 examples, moving-average loss 0.21, train accuracy 0.73
1,839,900 examples, moving-average loss 0.21, train accuracy 0.73
1,841,500 examples, moving-average loss 0.21, train accuracy 0.73
1,843,100 examples, moving-average loss 0.22, train accuracy 0.73
1,844,700 examples, moving-average loss 0.21, train accuracy 0.73
1,846,300 examples, moving-average loss 0.21, train accuracy 0.73
1,847,900 examples, moving-average loss 0.22, train accuracy 0.73
1,849,500 

2,025,500 examples, moving-average loss 0.20, train accuracy 0.73
2,027,100 examples, moving-average loss 0.21, train accuracy 0.73
2,028,700 examples, moving-average loss 0.21, train accuracy 0.73
2,030,300 examples, moving-average loss 0.21, train accuracy 0.73
2,031,900 examples, moving-average loss 0.21, train accuracy 0.73
2,033,500 examples, moving-average loss 0.21, train accuracy 0.73
2,035,100 examples, moving-average loss 0.21, train accuracy 0.73
2,036,700 examples, moving-average loss 0.22, train accuracy 0.73
2,038,300 examples, moving-average loss 0.21, train accuracy 0.73
2,039,900 examples, moving-average loss 0.21, train accuracy 0.73
2,041,500 examples, moving-average loss 0.21, train accuracy 0.73
2,043,100 examples, moving-average loss 0.21, train accuracy 0.73
2,044,700 examples, moving-average loss 0.22, train accuracy 0.73
2,046,300 examples, moving-average loss 0.20, train accuracy 0.73
2,047,900 examples, moving-average loss 0.21, train accuracy 0.73
2,049,500 

2,225,500 examples, moving-average loss 0.20, train accuracy 0.73
2,227,100 examples, moving-average loss 0.20, train accuracy 0.73
2,228,700 examples, moving-average loss 0.19, train accuracy 0.73
2,230,300 examples, moving-average loss 0.20, train accuracy 0.73
2,231,900 examples, moving-average loss 0.20, train accuracy 0.73
2,233,500 examples, moving-average loss 0.20, train accuracy 0.73
2,235,100 examples, moving-average loss 0.20, train accuracy 0.73
2,236,700 examples, moving-average loss 0.23, train accuracy 0.73
2,238,300 examples, moving-average loss 0.22, train accuracy 0.73
2,239,900 examples, moving-average loss 0.21, train accuracy 0.73
2,241,500 examples, moving-average loss 0.21, train accuracy 0.73
2,243,100 examples, moving-average loss 0.19, train accuracy 0.73
2,244,700 examples, moving-average loss 0.21, train accuracy 0.73
2,246,300 examples, moving-average loss 0.20, train accuracy 0.73
2,247,900 examples, moving-average loss 0.22, train accuracy 0.73
2,249,500 

2,425,500 examples, moving-average loss 0.20, train accuracy 0.74
2,427,100 examples, moving-average loss 0.20, train accuracy 0.74
2,428,700 examples, moving-average loss 0.20, train accuracy 0.74
2,430,300 examples, moving-average loss 0.22, train accuracy 0.74
2,431,900 examples, moving-average loss 0.20, train accuracy 0.74
2,433,500 examples, moving-average loss 0.22, train accuracy 0.74
2,435,100 examples, moving-average loss 0.21, train accuracy 0.74
2,436,700 examples, moving-average loss 0.22, train accuracy 0.74
2,438,300 examples, moving-average loss 0.19, train accuracy 0.74
2,439,900 examples, moving-average loss 0.19, train accuracy 0.74
2,441,500 examples, moving-average loss 0.21, train accuracy 0.74
2,443,100 examples, moving-average loss 0.20, train accuracy 0.74
2,444,700 examples, moving-average loss 0.20, train accuracy 0.74
2,446,300 examples, moving-average loss 0.21, train accuracy 0.74
2,447,900 examples, moving-average loss 0.22, train accuracy 0.74
2,449,500 

KeyboardInterrupt: 

## 5. Evaluate the Model 

In [142]:
test_batches = 1
test_accuracy = 0.0
test_pred_y = []

for (tx, ty) in utils.multi_batch_generator(train_params['batch_size'], \
                                        ds.padded_test_features, ds.test_ord_labels):

    batch_accuracy, pred_max = sess.run([accuracy_, pred_max_], feed_dict={X: tx, Y: ty})

    test_batches +=1
    test_accuracy += batch_accuracy
    test_pred_y.append(pred_max.tolist())

test_accuracy = test_accuracy/test_batches
print("Test accuracy:{}".format(test_accuracy))

pred_y = [y for x in test_pred_y for y in x]

Test accuracy:0.7119574175981375


In [143]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ds.test_labels, pred_y)

array([[ 7718,  2946,   257,    35],
       [ 1593,  7844,  3351,   243],
       [  224,  3233, 16614,  8606],
       [   65,   561,  8754, 41835]])

In [2]:
#op = sess.graph.get_operations()
#[m.values() for m in op]

